## Exercise: Integrating Dev Tooling into Git

1. Fork the Repository at <url will be put here later>.
2. Clone your forked repo to a local repository.
3. Set up 5 checks, repeating the following procdure **One Checker Tool at a Time**:
    1. Choose a static analysis checking tool you'd like to add to the set of checks on the repo.
    2. Create and switch to a New Git feature branch named: `feature-<check-tool>`.
    3. In a file called [`.pre-commit-config.yaml`](https://github.com/pre-commit/pre-commit-hooks?tab=readme-ov-file#using-pre-commit-hooks-with-pre-commit) in the root directory, add the checker tool.
    4. Run the tool, and correct the code until it passes: `pre-commit run <check-tool> -v`.
    5. Run all the checks, and correct the code until it passes: `pre-commit run all`.
    6. Register the checks with the Git Hooks system: `pre-commit install`.
    7. Commit your changes, confirming that the checks run automatically.
    8. Merge the feature branch into the `main` branch, and delete the feature branch.
    9. Push to `main` in GitHub.

## What a Day